In [1]:
import sqlite3

db = sqlite3.connect('thermo.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS HIGH") # Convenient in case you want to start over
cursor.execute("DROP TABLE IF EXISTS LOW") # Convenient in case you want to start over
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE LOW (
                SPECIES_NAME TEXT NOT NULL,
                TLOW INTEGER NOT NULL,
                THIGH INTEGER NOT NULL,
                COEFF_1 INTEGER NOT NULL,
                COEFF_2 INTEGER NOT NULL,
                COEFF_3 INTEGER NOT NULL,
                COEFF_4 INTEGER NOT NULL,
                COEFF_5 INTEGER NOT NULL,
                COEFF_6 INTEGER NOT NULL,
                COEFF_7 INTEGER NOT NULL)''')

db.commit()
cursor.execute('''CREATE TABLE HIGH (
                SPECIES_NAME TEXT NOT NULL,
                TLOW INTEGER NOT NULL,
                THIGH INTEGER NOT NULL,
                COEFF_1 INTEGER NOT NULL,
                COEFF_2 INTEGER NOT NULL,
                COEFF_3 INTEGER NOT NULL,
                COEFF_4 INTEGER NOT NULL,
                COEFF_5 INTEGER NOT NULL,
                COEFF_6 INTEGER NOT NULL,
                COEFF_7 INTEGER NOT NULL)''')


db.commit() # Commit changes to the database



In [2]:
import xml.etree.cElementTree as ET
tree = ET.parse('thermo.xml')
root = tree.getroot()
speciesData = root.findall('speciesData')
species = speciesData[0].findall('species')
for s in species:
    species = s.get('name')
    thermo = s.find('thermo')
    nasa = thermo.findall('NASA')
    nasa1 = nasa[0]
    nasa2 = nasa[1]
    tlow1 = nasa1.get('Tmin')
    thigh1 = nasa1.get('Tmax')
    tlow2 = nasa2.get('Tmin')
    thigh2 = nasa2.get('Tmax')
    coef1 = nasa1.find('floatArray').text
    coef2 = nasa2.find('floatArray').text
    COEFF1, COEFF2, COEFF3, COEFF4, COEFF5, COEFF6, COEFF7 = coef1.strip().split(',')
    COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7 = coef2.strip().split(',')
    low_values = (species, tlow1, thigh1, COEFF1, COEFF2, COEFF3, COEFF4, COEFF5, COEFF6, COEFF7)
    high_values = (species, tlow2, thigh2, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
    cursor.execute('''INSERT INTO LOW
               (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                low_values)
    cursor.execute('''INSERT INTO HIGH
               (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                high_values)
    db.commit()

In [3]:
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [4]:
low_cols = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
query = '''SELECT * FROM LOW'''
viz_tables(low_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,300,1000,2.569421,-8.597411e-05,4.194846e-08,-1.001778e-11,1.228337e-15,29217.579100,4.784339
1,O2,300,1000,3.282538,1.483088e-03,-7.579667e-07,2.094706e-10,-2.167178e-14,-1088.457720,5.453231
2,H,300,1000,2.500000,-2.308430e-11,1.615619e-14,-4.735152e-18,4.981974e-22,25473.659900,-0.446683
3,H2,300,1000,3.337279,-4.940247e-05,4.994568e-07,-1.795664e-10,2.002554e-14,-950.158922,-3.205023
4,OH,300,1000,3.092888,5.484297e-04,1.265052e-07,-8.794616e-11,1.174124e-14,3858.657000,4.476696
5,H2O,300,1000,3.033992,2.176918e-03,-1.640725e-07,-9.704199e-11,1.682010e-14,-30004.297100,4.966770
6,HO2,300,1000,4.017211,2.239820e-03,-6.336581e-07,1.142464e-10,-1.079085e-14,111.856713,3.785102
7,H2O2,300,1000,4.165003,4.908317e-03,-1.901392e-06,3.711860e-10,-2.879083e-14,-17861.787700,2.916157


In [5]:
high_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
query = '''SELECT * FROM HIGH'''
viz_tables(high_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,1000,5000,3.168267,-3.279319e-03,6.643064e-06,-6.128066e-09,2.112660e-12,29122.259200,2.051933
1,O2,1000,5000,3.782456,-2.996734e-03,9.847302e-06,-9.681295e-09,3.243728e-12,-1063.943560,3.657676
2,H,1000,5000,2.500000,7.053328e-13,-1.995920e-15,2.300816e-18,-9.277323e-22,25473.659900,-0.446683
3,H2,1000,5000,2.344331,7.980521e-03,-1.947815e-05,2.015721e-08,-7.376118e-12,-917.935173,0.683010
4,OH,1000,5000,3.992015,-2.401318e-03,4.617938e-06,-3.881133e-09,1.364115e-12,3615.080560,-0.103925
5,H2O,1000,5000,4.198641,-2.036434e-03,6.520402e-06,-5.487971e-09,1.771978e-12,-30293.726700,-0.849032
6,HO2,1000,5000,4.301798,-4.749121e-03,2.115829e-05,-2.427639e-08,9.292251e-12,294.808040,3.716662
7,H2O2,1000,5000,4.276113,-5.428224e-04,1.673357e-05,-2.157708e-08,8.624544e-12,-17702.582100,3.435051
